### Multimodel stats

For now: mean, max, min, standard deviation

In [2]:
%matplotlib inline

import xarray as xr
import intake
import pandas as pd
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

# util.py is in the local directory
# it contains code that is common across project notebooks
# or routines that are too extensive and might otherwise clutter
# the notebook design
import util 

/srv/conda/envs/notebook/lib/python3.7/site-packages/intake/source/discovery.py:136: FutureWarning: The drivers ['stac-catalog', 'stac-collection', 'stac-item'] do not specify entry_points and were only discovered via a package scan. This may break in a future release of intake. The packages should be updated.
  FutureWarning)


### FUNCTION INFO:

Find mean, min, and max across a list of models over the globe (default; lat-lon averaged) or over a certain region (a lat-lon average)

input: 

    models = list of the models with dimension (time, lon, lat) 
    
    loc = [lonmin, lonmax, latmin, latmax] (assumes all if not given)
    
*** assumes all models have the same length of times and that the dimension is called "time" ***

*** lat/lon are chosen using the nearest values in the dataset ***

returns:
    
    mean [dimensions of time]
    min [dimensions of time]
    max [dimensions of time]


In [10]:
def multimodel(models,loc=None):
    
    ntime = len(models[0].time)
    nmod = len(models)
    print('number time steps: ' + str(ntime))
    print('number of models: ' + str(nmod))
    
    tempdata = np.empty((nmod,ntime))
    
    ### average the data over the lat-lon range you need:###
    for m in range(nmod):
        model = models[m]
        
        # global mean:
        if loc == None:
            tempdata[m,:] = model.mean(dim=['lat','lon'])
            
        # specific location:
        else:
            if len(loc) != 4:
                raise Exception('Must enter location in the form [minlon, maxlon, minlat, maxlat]!')
            lonmin = loc[0]
            lonmax = loc[1]
            latmin = loc[2]
            latmax = loc[3]
            
            tempdata[m,:] = model.sel(lat=slice(latmin,latmax),method='nearest').sel(lon=slice(lonmin,lonmax),method='nearest').mean()
     
    
    ### multimodel mean and spread:###        
    mm_mean = np.nanmean(tempdata,axis=0)
    mm_min = np.min(tempdata,axis=0)
    mm_max = np.max(tempdta,axis=0)
    
    return [mm_mean,mm_min,mm_max]

In [11]:
# data to test: